In [131]:
import pennylane as qml
import numpy as np
import qiskit     
from qiskit.quantum_info import Pauli
from scipy.optimize import minimize
from qiskit import QuantumCircuit, execute, Aer
from qiskit.quantum_info import Statevector

In [137]:
def circuit(theta):
    # Create quantum circuit
    circ = QuantumCircuit(3)
    circ.h(range(3)) 
    for i, t in enumerate(theta[:3]):
        circ.p(t, i)
    circ.h(range(1, 3))  # Corrected the range here
    for i, t in enumerate(theta[3:]):
        circ.crx(t, i, i+1)  # Adjusted indices here
    backend = Aer.get_backend('statevector_simulator')
    job = execute(circ, backend)
    result = job.result()
    initial_state = result.get_statevector(circ)
    rho = np.outer(initial_state, np.conj(initial_state))
    return rho

In [138]:
def hamiltonian():
    # Create Identity and Pauli-Z operators
    I_ = Pauli('I').to_matrix()
    Z_ = Pauli('Z').to_matrix()
    Y_ = Pauli('Y').to_matrix()
    X_ = Pauli('X').to_matrix()

    # Create tensor products
    pauli_1 = np.kron(X_, np.kron(X_, I_))
    pauli_2 = np.kron(I_, np.kron(X_, X_))
    pauli_3 = np.kron(Y_, np.kron(Y_, I_))
    pauli_4 = np.kron(I_, np.kron(Y_, Y_))
    pauli_5 = np.kron(Z_, np.kron(Z_, I_))
    pauli_6 = np.kron(I_, np.kron(Z_, Z_))
   
    # Create Hamiltonian
    H = (pauli_1 + pauli_2 + pauli_3 + pauli_4 + pauli_5 + pauli_6)
    return H

In [139]:
def objective_function(theta):
    rho = circuit(theta)
    H = hamiltonian()
    expectation = np.trace(np.dot(H, rho))
    return expectation

In [140]:
# Initial guess for theta
initial_theta = np.random.rand(5) * 2 * np.pi

# Minimize the objective function
result = minimize(objective_function, initial_theta, method='Nelder-Mead')

# Optimal values of theta
optimal_theta = result.x

# Minimum expectation value
min_expectation = result.fun

In [141]:
print("Optimal theta:", optimal_theta)
print("Minimum expectation value:", min_expectation)

Optimal theta: [1.57080909 2.35621264 2.35617182 4.71233105 4.71244442]
Minimum expectation value: -3.8284271233663443
